In [1]:
#pip install torch torchvision numpy facenet-pytorch opencv-python


In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import os
import cv2
from tqdm import tqdm

In [2]:
# Hyperparameters
num_classes = 31  # Update based on your dataset
batch_size = 32
learning_rate = 0.001
num_epochs = 5

In [3]:
# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# Load data
data_dir = 'dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes



In [5]:
# Print dataset details
for phase in ['train', 'test']:
    print(f"Dataset '{phase}' contains {dataset_sizes[phase]} samples.")
    for class_name in class_names:
        num_samples = len([1 for _, label in image_datasets[phase].samples if image_datasets[phase].class_to_idx[class_name] == label])
        print(f" - {class_name}: {num_samples} samples")

Dataset 'train' contains 2562 samples.
 - Akshay Kumar: 50 samples
 - Alexandra Daddario: 92 samples
 - Alia Bhatt: 79 samples
 - Amitabh Bachchan: 74 samples
 - Andy Samberg: 92 samples
 - Anushka Sharma: 68 samples
 - Billie Eilish: 98 samples
 - Brad Pitt: 120 samples
 - Camila Cabello: 87 samples
 - Charlize Theron: 78 samples
 - Claire Holt: 96 samples
 - Courtney Cox: 80 samples
 - Dwayne Johnson: 61 samples
 - Elizabeth Olsen: 71 samples
 - Ellen Degeneres: 75 samples
 - Henry Cavill: 106 samples
 - Hrithik Roshan: 101 samples
 - Hugh Jackman: 112 samples
 - Jessica Alba: 108 samples
 - Kashyap: 30 samples
 - Lisa Kudrow: 70 samples
 - Margot Robbie: 72 samples
 - Marmik: 32 samples
 - Natalie Portman: 105 samples
 - Priyanka Chopra: 102 samples
 - Robert Downey Jr: 113 samples
 - Roger Federer: 77 samples
 - Tom Cruise: 58 samples
 - Vijay Deverakonda: 115 samples
 - Virat Kohli: 49 samples
 - Zac Efron: 91 samples
Dataset 'test' contains 2562 samples.
 - Akshay Kumar: 50 sampl

In [6]:
#Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
# Load a pre-trained InceptionResnetV1 model and modify the final layer
model = InceptionResnetV1(classify=True, num_classes=31)
model = model.to(device)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    model.load_state_dict(best_model_wts)
    return model

In [10]:
# Train and evaluate
model = train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs)



Epoch 0/4
----------
train Loss: 3.4985 Acc: 0.0492
test Loss: 3.2533 Acc: 0.1003

Epoch 1/4
----------
train Loss: 3.3248 Acc: 0.0859
test Loss: 3.0497 Acc: 0.1483

Epoch 2/4
----------
train Loss: 3.1692 Acc: 0.1183
test Loss: 2.8758 Acc: 0.1995

Epoch 3/4
----------
train Loss: 2.9941 Acc: 0.1608
test Loss: 2.8592 Acc: 0.1885

Epoch 4/4
----------
train Loss: 2.8264 Acc: 0.1768
test Loss: 2.5331 Acc: 0.2685



In [10]:
# Save the model
torch.save(model.state_dict(), 'face_recognition_model.pth')

In [11]:
def recognize_faces(image_path, model, mtcnn, device):
    image = Image.open(image_path)
    boxes, _ = mtcnn.detect(img=image)
    
    if boxes is None:
        print("No faces detected.")
        return []

    faces = []
    for box in boxes:
        face = image.crop((box[0], box[1], box[2], box[3]))
        face = transforms.functional.to_tensor(face)
        face = transforms.functional.resize(face, (160, 160))
        face = transforms.functional.normalize(face, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        faces.append(face)

    faces = torch.stack(faces).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(faces)
        _, preds = torch.max(outputs, 1)

    recognized_faces = [class_names[pred] for pred in preds]
    return recognized_faces, boxes


In [12]:
# Load the model for inference

model = InceptionResnetV1(classify=True, num_classes=31)
model = model.to(device)

model.load_state_dict(torch.load('face_recognition_model.pth'))
model = model.to(device)


In [17]:
# Recognize faces
image_path = 'dataset/test/Alia Bhatt/Alia Bhatt_1.jpg'
mtcnn = MTCNN(keep_all=True, device=device)
recognized_faces, boxes = recognize_faces(image_path, model, mtcnn, device)
for face, box in zip(recognized_faces, boxes):
    print(f'Recognized person: {face}, Box: {box}')

Recognized person: Hrithik Roshan, Box: [228.5430145263672 103.81326293945312 378.9773254394531 274.3039855957031]


In [ ]:
def recognize_faces(image_path, model, mtcnn, device):
    image = Image.open(image_path)
    boxes, _ = mtcnn.detect(img=image)
    
    if boxes is None:
        print("No faces detected.")
        return []

    faces = []
    for box in boxes:
        face = image.crop((box[0], box[1], box[2], box[3]))
        face = transforms.functional.to_tensor(face)
        face = transforms.functional.resize(face, (160, 160))
        face = transforms.functional.normalize(face, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        faces.append(face)

    faces = torch.stack(faces).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(faces)
        _, preds = torch.max(outputs, 1)

    recognized_faces = [class_names[pred] for pred in preds]
    return recognized_faces, boxes


In [ ]:
import os
parent_path='dataset'
test_path='test'
train_path='train'
cropped_dataset_path='cropped_dataset'
if not os.path.exists(cropped_dataset_path):
    os.mkdir(cropped_dataset_path)
mtcnn = MTCNN(keep_all=True, device=device)
for datadirectory in os.listdir(parent_path):
    path_to_person=os.path.join(parent_path,datadirectory)
    for person_name in os.listdir(path_to_person):
        person_images_path=os.path.join(path_to_person,person_name)
        all_images_path=os.listdir(person_images_path)
        images_for_test=all_images_path[0:10]
        images_for_train=all_images_path[10:]
        parent_new_path=os.path.join(cropped_dataset_path,'train')
        if not os.path.exists(parent_new_path):
            os.mkdir(parent_new_path)
        for images_path in images_for_test:

            image_path=os.path.join(person_images_path,images_path)
            image = Image.open(image_path)
            boxes, _ = mtcnn.detect(img=image)
            if boxes is None:
                print("No faces detected.")
            else:
                faces = []
                for box in boxes:
                    face = image.crop((box[0], box[1], box[2], box[3]))
                    face = transforms.functional.to_tensor(face)
                    face = transforms.functional.resize(face, (160, 160))
                    face = transforms.functional.normalize(face, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    faces.append(face)
                for i in range(len(faces)):
                    new_path=os.path.join(parent_new_path,person_name)
                    if not os.path.exists(new_path):
                        os.mkdir(new_path)
                    new_path=os.path.join(new_path,f'{person_name}_{i}.jpg')
                    print(new_path)
                    cv2.imwrite(new_path,np.array(face[i]))

cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Akshay Kumar\Akshay Kumar_0.jpg
cropped_dataset\train\Alexandra Daddario\Alexandra Daddario_0.jpg
cropped_dataset\train\Alexandra Daddario\Alexandra Daddario_0.jpg
cropped_dataset\train\Alexandra Daddario\Alexandra Daddario_0.jpg
cropped_dataset\train\Alexandra Daddario\Alexandra Daddario_0.jpg
cropped_dataset\train\Alexandra Daddario\Alexandra Daddario_0.jpg
